# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Applications of Linear Algebra

The reason we had you review linear algebra is because it is an incredibly useful and important tool in modern data analysis. To convince you of this fact, this notebook presents two of the classic linear algebra problems as they might arise in data analysis: solving linear systems of equations and eigenvalue problems (or "eigenproblems").

## Example 1, solving linear systems: Linear regression

Let's start by loading a dataset and motivating the problem of modeling it using linear regression. This dataset is stored in a CSV file called `uscrime.csv`, which contains crime statistics from 1960. It is described here: http://www.statsci.org/data/general/uscrime.html

In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
df = pd.read_csv ('uscrime.csv', skiprows=1)
display (df.head ())

Each row of this dataset is a US State. The columns are described here: http://www.statsci.org/data/general/uscrime.html

Here are some summary statistics for each column:

In [ ]:
df.describe ()

Next, let's visualize the relationships among a subset of the columns:

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
# Look at a few relationships
sns.pairplot (df[['Crime', 'Wealth', 'Ed', 'U1']])

Suppose we wish to build a model of some quantity, called the _response_ variable, given some other set of _predictor variables_, or just _predictors_. In this US crimes dataset, the response might be the crime rate (`Crime`), which we wish to predict from the predictors of income (`Wealth`), education (`Ed`), and the unemployment rate of young males (`U1`). That is, we wish to determine a function $f()$ such that

$$\mathtt{Crime} \approx f(\mathtt{Wealth}, \mathtt{Ed}, \mathtt{U1}).$$

Of course, there are an infinite number of choices of the function $f()$! To make the problem more tractable, we typically guess the form of $f()$ and _parameterize_ it by some unknown set of parameters, say, $\theta_1$, $\theta_2$, ..., $\theta_n$:

$$
  \mathtt{Crime} \approx f(\mathtt{Wealth}, \mathtt{Ed}, \mathtt{U1}; \theta_1, \theta_2, \ldots, \theta_n).
$$

Then, rather than trying all possible functions $f()$, we look for the parameters $\theta$ that best fit the given data.

> In _statistical modeling_, we typically choose functions $f()$ that have a probabilistic interpretation.

**Linear (regression) models.** An especially simple form for $f()$ is one that is _linear_ in the model parameters, e.g.,

$$
  \mathtt{Crime} \approx \theta_0 + (\theta_1 \cdot \mathtt{Wealth}) + (\theta_2 \cdot \mathtt{Ed}) + (\theta_3 \cdot \mathtt{U1}).
$$

In this example, the model uses the parameters ($\theta_i$) as _linear_ weights for each predictor. Besides these weights, the model also includes a "baseline" parameter, $\theta_0$, that is independent of any predictor.

> How would you interpret $\theta_0$ in this example?

**The linear algebra problem.** Armed with this model, the next question is what values of parameters to use so that the model "best fits" the observations (i.e., rows of the US Crimes tibble)?

Let's start by generalizing the problem and developing a natural, albeit _ad hoc_, procedure to determine the model parameters.

Suppose there are $m$ observations in total and consider the $i$-th observation. Denote its response by $y_i$ and its predictors by $\{x_{i,1}, x_{i,2}, \ldots, x_{i,n}\}$. If we want to consider linear models of these data, then we are guessing that

$y_i \approx \theta_0 + x_{i,1} \theta_1 + x_{i,2} \theta_2 + \cdots + x_{i,n} \theta_n$

for all $i$.

Let's adopt the convention that the observations are numbered starting at 0 and going up to $m-1$. Since we want the above model to hold for all observations, we can write everything as the system of linear equations,

$$
    \left(
      \begin{array}{c}
      y_0 \\
      y_1 \\
      \vdots \\
      y_{m-1}
      \end{array}
     \right)
    \approx
    \left(
       \begin{array}{ccccc}
         1. & x_{0,1}   & x_{0,2}   & \ldots & x_{0,n} \\
         1. & x_{1,1}   & x_{1,2}   & \ldots & x_{1,n} \\
            &           & \cdots    &        &         \\
         1. & x_{m-1,1} & x_{m-1,2} & \ldots & x_{m-1,n}
       \end{array}
     \right)
     \left(\begin{array}{c}
       \theta_0 \\
       \theta_1 \\
       \vdots \\
       \theta_n
     \end{array}\right).
$$

To ensure we number all our variables consistently, let's adopt one more convention. Let's create a "dummy" variable, $x_{i,0} \equiv 1$. Then, each equation becomes

$y_i \approx x_{i,0} \theta_0 + x_{i,1} \theta_1 + x_{i,2} \theta_2 + \cdots + x_{i,n} \theta_n,$

and the entire first column of the matrix changes to

$$
    \left(
      \begin{array}{c}
      y_0 \\
      y_1 \\
      \vdots \\
      y_{m-1}
      \end{array}
     \right)
    \approx
    =
    \left(
       \begin{array}{ccccc}
         x_{0,0}   & x_{0,1}   & x_{0,2}   & \ldots & x_{0,n} \\
         x_{1,0}   & x_{1,1}   & x_{1,2}   & \ldots & x_{1,n} \\
                   &           & \cdots    &        &         \\
         x_{m-1,0} & x_{m-1,1} & x_{m-1,2} & \ldots & x_{m-1,n}
       \end{array}
     \right)
     \left(\begin{array}{c}
       \theta_0 \\
       \theta_1 \\
       \vdots \\
       \theta_n
     \end{array}\right),
$$

which can in turn be written more compactly as

$$
    y \approx X \theta.
$$

Note that $X$ is an $m \times (n+1)$ matrix and $\theta$ is a vector of length $n+1$.

**Formalizing the problem of estimating $\theta$.** The model is necessarily approximate. Indeed, typically $m \gg n$, in which case we say that the _exact_ system, $y = X \theta$, is _overdetermined_ and the $\theta$ for such a system does not exist. So then how do we determine an approximate $\theta$ that is "close enough" or "good enough?"

Let's write down a criterion for choosing $\theta$, which will lead (eventually) to an algorithmic procedure.

> Our reasoning below will be _ad hoc_, but it will turn out that this procedure corresponds to a probabilistic interpretation of the problem! (We won't cover that fact in this class but it's likely you have taken or will soon take a statistics class.)

Suppose we have a guess, $\theta$. Then the error in this guess will be the _residual vector_,

$$
  r \equiv X\theta - y.
$$

The residual vector $r$ measures the difference between the model's estimated response, $X \theta$, and the true response, $y$. You can also think of $r$ as a measure of the "error" in the model compared to reality.

In scalar form, we would say that for all observations $i$,

$$
  r_i = \sum_{j=0}^{n} x_{i,j} \theta_j - y_j.
$$

Each entry $r_i$ is the _residual value_ for the $i$-th observation. We will sometimes write "residual" as a shortcut for either the residual vector or a residual value, depending on the context.

Eventually, we want to choose values of the parameters $\theta$ that minimize the residual $r$ in some way. Let's measure the size of $r$ by taking the square of the length of Euclidean length of the residual vector. We will call this quantity, $\|r\|_2^2$, the _residual squared error_. Another way to interpret $\|r\|_2^2$ is as the _sum of squared residuals (values)_,

$$
  \|r\|_2^2 = \sum_{i=0}^{m-1} r_i^2.
$$

> If you've taken a statistics or regression class, you may recognize the residual squared error as the "sum of squares error."

A natural goal is to choose $\theta$ to _minimize_ the residual squared error. This choice of measure, since it squares residuals, has the effect of giving higher weight to larger residuals. 

To formalize this minimization problem, observe that the residual square error may be rewritten as

$$
\begin{eqnarray}
    \|r\|_2^2 & = & \|X \theta - y\|_2^2.
\end{eqnarray}
$$

Thus, we have arrived at the formal problem we wish to solve. Let

$$
  g(\theta) \equiv \|X \theta - y\|_2^2.
$$

The problem is to compute the value $\hat{\theta}$ such that

$$
  \hat{\theta} = \arg\!\min_{\theta} g(\theta).
$$

This problem is sometimes referred to as the _linear least squares problem_.

The purpose of this discussion so far is to show that we will need tools from linear algebra, as well as calculus and (mathematical) optimization, in order to solve even the most basic data analysis problems. These details appear next; but for now, you can also defer that discussion to a future lab/class and instead move on to the second motivating example, which gives rise to an eigenvalue problem.

**Gradients.** In elementary calculus, when you need to locate the maximum or minimum value $x$ to some function $f(x)$, where $x$ is a scalar (not vector) value, you took the derivative of $f$ with respect to $x$, set it to 0, and then solved for $x$.

> You also needed an extra analysis step to show whether that solution, call it $x_*$, gave you a (local) maximum, minimum, or saddle point, but let's ignore that for the moment.

To find the minimizing $\hat{\theta}$, we need to do the moral equivalent of taking a "vector derivative" with respect to $\theta$, setting that derivative to 0, and then solving for $\theta$. The right mathematical tool is the _gradient_.

Given a _scalar_ function $f(x)$, where $x$ is a _vector_, the function's gradient with respect to $x$ is denoted by $\nabla_x f(x)$, which is a _vector_ whose $k$-th entry is the partial derivative of $f(x)$ with respect to $x_k$. That is,

$$
\nabla_x f(x) \equiv
  \left(\begin{array}{c}
    \frac{\partial f}{\partial x_0} \\
    \frac{\partial f}{\partial x_1} \\
    \vdots \\
    \frac{\partial f}{\partial x_{n-1}}
  \end{array}\right).
$$

> _Example._ Let $x^T \equiv (x_0, x_1)$ and $f(x) \equiv x^T x = x_0^2 + x_1^2$. Then,
> 
> $$
  \nabla_x f(x)
  = \left(\begin{array}{c}
      \frac{\partial f}{\partial x_0} \\
      \frac{\partial f}{\partial x_1}
    \end{array}\right)
  = \left(\begin{array}{c}
      \frac{\partial}{\partial x_0}(x_0^2 + x_1^2) \\
      \frac{\partial}{\partial x_1}(x_0^2 + x_1^2)
    \end{array}\right)
  = \left(\begin{array}{c}
      2 x_0 \\
      2 x_1
    \end{array}\right)
  = 2 \left(\begin{array}{c}
        x_0 \\
        x_1
      \end{array}\right)
  = 2 x.
$$
>
> Notice that this result is consistent with what you would expect had $x$ been a scalar, so that $f(x) = x^2$ and its first (ordinary) derivative $\frac{df}{dx} = 2x$.

You should verify the following identities related to the gradient. Below, take $x$ and $y$ to be vectors and $M$ to be a matrix.

1. $\nabla_x (x^T y) = y$.
2. $\nabla_x (x^T x) = 2x$. (That is, generalize the example above to an $n$-vector.)
3. $\nabla_x (x^T M x) = (M + M^T)x$.

**Expanding the residual squared error.** Here's are some more handy vector identities, which you should verify:

1. $\|x\|_2^2 = x^T x$
2. $x^T y = y^T x$

**Computing $\hat{\theta}$.** Armed with the gradient, we are now ready to minimize $g(\theta)$.

$$
  \begin{eqnarray}
  \nabla_\theta g(\theta)
    & = & \nabla_\theta \|X\theta - y\|_2^2
    & = & \nabla_\theta \left( \theta^T X^T X \theta - 2 \theta^T X^T y + y^T y \right)
    & = & 2 (X^T X \theta - X^T y).
  \end{eqnarray}
$$

In other words, the minimizing parameters, $\hat{\theta}$, which would cause this gradient to go to zero, are the solution of $X^T X \hat{\theta} = X^T y$. This system is known as the _normal equations_.

**Algorithms?** The normal equations immediately suggest the following algorithm.

1. Form $A \equiv X^T X$.
2. Form $b \equiv X^T y$.
3. Solve $A\hat{\theta} = b$.

While this procedure _can_ work, we'll see in a future class why it's usually not a great algorithm. For now, it does motivate the need to solve a linear system of equations, which was the main goal.

## Example 2, an eigenproblem: Ranking web pages

Imagine a user who is looking for information on the web, having only some search terms, e.g., "`free donuts.`" There may be many pages that match these terms; in what order should the search engine return matching pages to the user?

**A search framework.** Here is one two-phase framework for search. The first phase is _offline_, meaning it happens _before_ any searches take place. The other phase is _online_, meaning it happens _at search time_, when the user's search terms are known.

The particular method we will discuss was developed by Google as their original search algorithm (late 1990s). It is an instance of this two-phase framework. The offline phase _precomputes_ a global ranking of all web pages, given some snapshot of the web. The online phase uses the search terms to filter the ranked list, and it returns those pages in the global rank order.

**The high-level PageRank idea.** The PageRank algorithm is the offline phase. It is based on a probabilistic model of how a hypothetical user might surf the web, in the absence of any specific query.

Here is a somewhat informal description of the model; the details follow in the next section.

Suppose there are $n$ web pages, represented by a _vertex set_ $V = \{0, 1, 2, \ldots, n-1\}$. The pages link to one another; let $E$ denote the _edge set_, which is a set of pairs $(i, j)$ indicating that page $i$ points to page $j$. We will sometimes write this link relationship as $i \rightarrow j$. This representation is also known as a [_directed graph_](https://en.wikipedia.org/wiki/Directed_graph) representation.

Next, consider a "random web surfer." This surfer visits web pages one at a time, according to the following stochastic process.

1. At each time step $t \geq 0$, the surfer visits a page. Further assume that the surfer's choice of page at time $t+1$ depends only on the page visited at time $t$. (This assumption makes this process a [_discrete-time Markov chain_](https://en.wikipedia.org/wiki/Markov_chain).)
2. Initially, at time $t=0$, the web surfer starts on a random page.
3. Suppose the surfer is on page $i$ at time $t$. With probability $\alpha$, the surfer will decide to follow a link going from $i$ to some new page $j$.
4. At time $t$, the surfer might instead decide, with probability $1-\alpha$, to _jump_ to some page $j$. Page $j$ is _not necessarily_ directly connected to $i$.

As time proceeds, the surfer jumps from page to page, sometimes hitting a page it has already visited, and sometimes jumping to an entirely different part of the web graph. Now imagine that the surfer surfs for an _infinitely long time_; what is the probability that the surfer will be on any given page? If this probability distribution can be calculated, then the PageRank idea is to use the distribution to rank the web pages: the most likely page for the surfer to land on is the top-ranked page, the next most likely page is the second-ranked page, and so on.

### Mathematical Details ###

To fully specify the process outlined above, we need to pin down a few more details. We will do so using probabilistic statements, which it then turns out we will be able to write down succinctly in the language of linear algebra.

**Connectivity matrix.** Let's start by representing the graph by a matrix $G \equiv (g_{ij})$, where the $(i, j)$ entry, $g_{ij}$, is 1 if there is a link $i \rightarrow j$, and 0 otherwise.

**Probability state vector.** Next, let $x(t)$ be a (column) vector that represents the probabilities that the surfer is on each page at time $t$. That is,

$x(t) \equiv \left(\begin{array}{c} x_0(t) \\ x_1(t) \\ \vdots \\ x_{n-1}(t) \end{array}\right)$,

where $x_i(t)$ is the probability that the surfer is on page $i$ at time $t$. Since the surfer must always be on some page, these probabilities must sum to 1: $\sum_{i=0}^{n-1} x_i(t) = 1$.

**Surfing process.** At time $t=0$, suppose that the surfer is equally likely to start on any page. Then, $x_i(0) = \frac{1}{n}$.

Now suppose that the surfer is on page $i$ at time $t$. What page will the surfer visit at time $t+1$? Recall that there are two scenarios in our high-level model: follow an out-link or jump to another page.

_Scenario 1._ If the surfer decides to follow an out-link, which one will it choose?

Let's assume the surfer picks an outgoing link uniformly at random. That is, if page $i$ has $d_i$ outgoing links, then let the probability of choosing an out-link be $\frac{1}{d_i}$. The value $d_i$ is also called the _out-degree_ of vertex $i$. It may be computed by summing each row of $G$, i.e., $d_i \equiv \sum_{j=0}^{n-1} g_{ij}$.

Thus, given the decision to follow an out-link starting from page $i$, the probability of moving to page $j$ is $p_{ij} \equiv \frac{g_{ij}}{d_i}$.

What if page $i$ _has no_ outgoing edges? There are several ways to handle this case. The simple one we will adopt is to _force_ it to have a self-edge, $i \rightarrow i$. In other words, the surfer has decided to follow an out-link but has nowhere to go; therefore, it stays put on page $i$. Mathematically, the self-edge means $d_i = g_{ii} = 1$ and $p_{ii} = 1$.

> Aside: This way of handling pages without outgoing edges differs from the way they are treated both in the original PageRank scheme and in Cleve Moler's notes, which we link to below. The original PageRank scheme simply removed these pages. By contrast, Moler assumes that when there is no outgoing edge, then the surfer jumps to any random page, just like the $1-\alpha$ case. In other words, Moler would set all $g_{ij} = 1$ for all $j$ when $i$ has no outgoing links.

Given all of the $g_{ij}$, including self-edges when needed, we can express all of these quantities in matrix notation:

$G \equiv \left( g_{ij} \right),
 \qquad D \equiv \left(\begin{array}{ccc} d_0 && \\ & \ddots & \\ && d_{n-1} \end{array}\right),
 \qquad P \equiv \left( p_{ij} \right) = D^{-1}G$

The matrix $P$ is sometimes called a [probability transition matrix](https://en.wikipedia.org/wiki/Stochastic_matrix). From the definitions above, you should convince yourself that every row of the matrix $P \equiv (p_{ij})$ sums to 1.

_Scenario 2._ If instead the surfer decides to jump to a random page, which one will it choose?

Again, let's assume the surfer jumps uniformly at random to any one of the $n$ pages, which includes itself and any outgoing links from the current page. Then, the probability of choosing any other page $j$ would be just $\frac{1}{n}$.

**Putting it all together.** We now have all the details we need to compute the probability of ending up on a page $i$ at time $t+1$ starting from some page $j$ at time $t$. This probability is, as a scalar formula,

$x_i(t+1) = \left[\alpha \cdot \sum_{j=0}^{n-1} p_{ji} x_j(t)\right] + \left[(1-\alpha) \cdot \frac{1}{n}\right].$

We can also write this more compactly in matrix notation. Let $u$ be a vector whose entries are all equal to 1. Then the above formula for all $i$ is the same as,

$x(t+1) = \alpha P^T x(t) + \frac{1 - \alpha}{n} u$.

From the definition of $P$, note that $P^T = G^T D^{-1}$.

Thus, the PageRank algorithm is the following: run the preceding iteration, starting at $t=0$, until $x(t)$ appears to stabilize (i.e., has reached steady-state) or, as is typically done, until some maximum number of time steps has been reached.

**Convergence?** At least one detail remains: how do we know that the state vector $x(t)$ will _ever_ reach a steady-state? Could the probabilities oscillate, diverge, or exhibit chaotic behavior?

The analysis to prove that a steady-state exists is somewhat involved, but the gist is the following. The formula to compute $x(t+1)$ from $x(t)$ can also be written as follows:

$x(t+1) = \hat{A} x(t)$,

where

$\hat{A} \equiv \alpha P^T + \frac{1-\alpha}{n} uu^T$. (You can convince yourself of this fact by first observing that $u^Tx(t)=1$.)

Thus, when we ask whether this process reaches steady-state, then we are effectively asking whether there is a vector $x$ such that $x = \hat{A} x$.

Like $P^T$ itself, the matrix $\hat{A}$ has the following properties, which you should verify.

* Its entries all lie between 0 and 1.
* The columns sum to 1.

From these facts, one may apply a theorem from linear algebra called the _Perron-Frobenius theorem_ and conclude that $x = \hat{A} x$ has a solution that is _both_ non-zero _and_ unique to within a scaling factor.

**Additional references on PageRank.** There are many other (and probably better) descriptions of the PageRank method out there on the web! We recommend these in particular, but if you find others, post on Piazza!

* Cleve Moler's notes on PageRank -- Moler's explanation is similar to what we discussed in class and what appears in these notes, but they are _not_ identical as these notes as explained below: [link](https://www.mathworks.com/moler/exm/chapters/pagerank.pdf)
* The original PageRank paper by Page et al.: [link](http://ilpubs.stanford.edu:8090/422/)

# Summary

The aim of this notebook was to show you how two fundamental problems of linear algebra---solving a linear system, $Ax=b$, and computing the eigenvalues and eigenvectors of a matrix $A$---arise in the context of practical data analysis. Armed with this motivation, we can now turn to the (hopefully fun) question of how to implement all this good stuff!